In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder # para realizar el Label Encoding 
from sklearn.preprocessing import OneHotEncoder  # para realizar el One-Hot Encoding

import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None

En el pair programming de hoy usaremos el set de datos que guardastéis en el pair programming de normalización y estandarización.

In [2]:
df=pd.read_csv('../datos/salaries_modelo.csv', index_col=0)
df.head(2) #Hacemos una vista previa de nuestro DataFrame

,salary_box,experience_level,employee_residence
0,4631.904560,Entry-Level,US
1,7311.922975,Mid-Level,US


Vuestro set de datos debería tener al menos una variable categórica, el objetivo del pair programming de hoy:

- Hacer una códificación de la/las variables categóricas que tengáis en vuestro set de datos.

Empezamos con la clasificación de nuestras variable categóricas según sean nominales, ordinarias o binarias:

    -  Ordinarias: experience_level
    -  Nomiales: company_location

Hacemos el encoding para variable categóricas ordinarias:

In [3]:
# Hacemos una funcion de mapeo

def encoder_map(df, columna, orden_valores):
    
    # nos creamos un diccionario vacío para hacer el map después. 
    ordinal_dict = {}
    
    # iteramos por nuestra lista de valores usando el enumerate que recordamos nos devolvía también el índice o posición de cada elemento
    for i, valor in enumerate(orden_valores):
        ordinal_dict[valor]=i+1 # le sumamos uno a la posición para no tener valores de 0.
        columna_nueva =columna + "map_funcion"

    # aplicamos el map
    df[columna_nueva] = df[columna].map(ordinal_dict)
    return df
    

In [4]:
## Experience_level:

df['experience_level'].unique()

array(['Entry-Level', 'Mid-Level', 'Senior', 'Executive Level'],
      dtype=object)

In [6]:
df = encoder_map(df, 'experience_level', ['Entry-Level', 'Mid-Level', 'Senior', 'Executive Level'])

In [7]:
df.head(2)

,salary_box,experience_level,employee_residence,experience_levelmap_funcion
0,4631.904560,Entry-Level,US,1
1,7311.922975,Mid-Level,US,2


Hacemos el endcoding para las variables categóricas nominales

- Recordad que lo primero que deberéis hacer es decidir su vuestras variables tienen o no orden, para que en función de esto uséis una aproximación u otra.

- Guardad el dataframe, donde deberíais tener las variables estadandarizas, normalizadas y codificadas en un csv para usarlo en el próximo pairprogramming